In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.io import loadmat, savemat
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import random

In [ ]:
p_num=39
a = []

for i in range(10):
    a.append(np.ceil((np.random.permutation(p_num)+1)/(p_num/5.0)).reshape(1,p_num))

a = np.array(a)
len(a)

In [ ]:
#normal_Y=np.array([29,35,45,55,65,83,90,104,106,108,110,112,126,137,140,151,154,163]);
#normal_M=np.array([15,26,42,44,53,64,89,105,160,161]);
#normal_O=np.array([1,10,12,13,14,18,20,41,62,77,133,153]);
 
#mtom_Y=np.array([25,47,50,56,59,63,92,111,51,58,80,94,102,114,144]);
#mtom_M=np.array([5,93,99,116,122,130,143,3,8,11,38,49,54,68,72,82,88,91,95,98,107,138]);
#mtom_O=np.array([2,4,6,7,16,96,100,118,125,142,145,21,67,101,103,115,121,127,136]);
 
#severe_Y=np.array([24,28,31,34,36,40,60,76,128,135]);
#severe_M=np.array([17,19,23,27,57,66,71,74,78,79,84,85,109,139,152]);
#severe_O=np.array([30,43,46,48,52,69,70,73,75,81,86,97,117,129,131,134,156,162]);

normal_Y=np.array([35,45,55,65,83,104,106,108,110,112,126,137,151,154,163]);
normal_M=np.array([15,26,42,44,53,64,89,105,160,161]);
normal_O=np.array([1,10,12,13,14,18,20,41,62,77,133,153]);
 
mtom_Y=np.array([25,47,50,56,59,63,92,111,51,58,94,102,114,144]);
mtom_M=np.array([5,99,116,122,130,8,11,38,49,68,82,91,95,98,138]);
mtom_O=np.array([2,4,7,96,118,125,142,21,67,101,115,127,136]);
 
severe_Y=np.array([24,28,31,34,36,40,60,76,128,135]);
severe_M=np.array([17,19,23,57,66,71,74,78,79,84,85,109,139,152]);
severe_O=np.array([30,43,46,69,70,73,75,86,97,117,129,131,134,156,162]);


control_num=np.array([35,45,55,65,83,104,106,108,110,112,126,137,151,154,163,
                      25,47,50,56,59,63,92,111,51,58,94,102,114,144,
                      24,28,31,34,36,40,60,76,128,135]);
Mid=np.array([15,26,42,44,53,64,89,105,160,161,
              5,99,116,122,130,8,11,38,49,68,82,91,95,98,138,
              17,19,23,57,66,71,74,78,79,84,85,109,139,152]);
Old=np.array([1,10,12,13,14,18,20,41,62,77,133,153,
              2,4,7,96,118,125,142,21,67,101,115,127,136,
              30,43,46,69,70,73,75,86,97,117,129,131,134,156,162]);

In [ ]:
num = 0
for k in range(len(a)):
    b = a[k]
    
    for m in range(1,6):
        index_test = np.where(b == m)
        index_train = np.where(b != m)
        
        test_set = np.empty((32,0))
        train_set = np.empty((32,0))
        test_set_label = np.empty((1,0))
        train_set_label = np.empty((1,0))
        
        test_set_M = np.empty((32,0))
        test_set_label_M = np.empty((1,0))
        test_set_O = np.empty((32,0))
        test_set_label_O= np.empty((1,0))
                
        train_set2 = np.empty((32,0))
        test_set_label2 = np.empty((1,0))
        
        test = index_test[1]
        train = index_train[1]
        
        print("{}번째 아우터 루프의 {}번째 테스트 인덱스".format(k, m))
        print(test)
        print("{}번째 아우터 루프의 {}번째 트레인 인덱스 ".format(k, m))
        print(train)
        print("\n")
        
        for n in test:
            test_data = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL.mat".format(control_num[n]))
            test_data = test_data['TOTAL']
            test_set = np.append(test_set, test_data, 1)
            
            test_label = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL_label.mat".format(control_num[n]))
            test_label = test_label['label_TOTAL']
            test_set_label = np.append(test_set_label, test_label, 1)
            
        for m in train:
            train_data = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL.mat".format(control_num[m]))
            train_data = train_data['TOTAL']
            train_set = np.append(train_set, train_data, 1)
            
            train_label = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL_label.mat".format(control_num[m]))
            train_label = train_label['label_TOTAL']
            train_set_label = np.append(train_set_label, train_label, 1)
            
        
        ind1=np.where(train_set_label==1)
        ind2=np.where(train_set_label==2)
        ind3=np.where(train_set_label==3)
        ind4=np.where(train_set_label==4)
        ind5=np.where(train_set_label==5)
        print([np.size(ind1[1]),np.size(ind2[1]),np.size(ind3[1]),np.size(ind4[1]),np.size(ind5[1])])
        st_min=np.min([np.size(ind1[1]),np.size(ind2[1]),np.size(ind3[1]),np.size(ind5[1])])
        ##st_num=np.where([np.size(ind1[1]),np.size(ind2[1]),np.size(ind3[1]),np.size(ind5[1])]==st_min)[0]
        ##print(st_num+1)

        st1=list(range(np.size(ind1[1]))); st1_pos=random.sample(st1,st_min); st1_p=ind1[1][st1_pos]
        st2=list(range(np.size(ind2[1]))); st2_pos=random.sample(st2,st_min); st2_p=ind2[1][st2_pos]
        st3=list(range(np.size(ind3[1]))); st3_pos=random.sample(st3,st_min); st3_p=ind3[1][st3_pos]
        st4_p=ind4[1]
        st5=list(range(np.size(ind5[1]))); st5_pos=random.sample(st5,st_min); st5_p=ind5[1][st5_pos]

        train_set2 = np.hstack((train_set[:,st1_p],train_set[:,st2_p],train_set[:,st3_p],train_set[:,st4_p],train_set[:,st5_p]))
        train_set_label2 = np.hstack((train_set_label[:,st1_p],train_set_label[:,st2_p],train_set_label[:,st3_p],train_set_label[:,st4_p],train_set_label[:,st5_p]))    

        train_set2 = np.transpose(train_set2)
        train_set_label2 = np.transpose(train_set_label2).ravel()
    
        test_set = np.transpose(test_set)
        test_set_label = np.transpose(test_set_label)
        
        param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 10]}
        clf_grid = GridSearchCV(SVC(kernel = 'rbf'), param_grid) 
        clf_grid.fit(train_set2, train_set_label2)
        C_best = clf_grid.best_params_["C"]
        gamma_best = clf_grid.best_params_["gamma"]

        model = SVC(kernel = 'rbf', C=C_best, gamma=gamma_best)
        model.fit(train_set2, train_set_label2)
        
        pred = model.predict(test_set)
        
        ##### M #########
        c = random.sample(range(len(normal_M)), len(test))
                
        for n in Mid[c]:
            test_data_M = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL.mat".format(n))
            test_data_M = test_data_M['TOTAL']
            test_set_M = np.append(test_set_M, test_data_M, 1)
            
            test_label_M = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL_label.mat".format(n))
            test_label_M = test_label_M['label_TOTAL']
            test_set_label_M = np.append(test_set_label_M, test_label_M, 1)
        
            
        test_set_M = np.transpose(test_set_M)
        test_set_label_M = np.transpose(test_set_label_M)
        pred_M = model.predict(test_set_M)
        
        ##### O #########
        c_1 = random.sample(range(len(normal_O)), len(test))
                
        for n in Old[c_1]:
            test_data_O = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL.mat".format(n))
            test_data_O = test_data_O['TOTAL']
            test_set_O = np.append(test_set_O, test_data_O, 1)
            
            test_label_O = loadmat("/user/chae2089/CWpaper/features_6/subject{}_PSD_TOTAL_label.mat".format(n))
            test_label_O = test_label_O['label_TOTAL']
            test_set_label_O = np.append(test_set_label_O, test_label_O, 1)
        
        test_set_O = np.transpose(test_set_O)
        test_set_label_O = np.transpose(test_set_label_O)
        pred_O = model.predict(test_set_O)
        
        
        ######### SAVE normal_Y
        result=confusion_matrix(test_set_label,pred)

        filename=("/user/chae2089/CWpaper/young_kfold_feature6/num_118/young/result_%d" %(num))
        np.savetxt(filename, result)
        
        report = classification_report(test_set_label, pred)
        report2=report.split()
        with open("/user/chae2089/CWpaper/young_kfold_feature6/num_118/young/report_{}.csv".format(num), 'w') as fp:
            for item in report2:
                fp.write("%s\n" % item)
                
        ######### SAVE normal_M    
        result_M=confusion_matrix(test_set_label_M,pred_M)

        filename=("/user/chae2089/CWpaper/young_kfold_feature6/num_118/mid/result_%d" %(num))
        np.savetxt(filename, result_M)
        
        report_M = classification_report(test_set_label_M, pred_M)
        report2_M=report_M.split()
        with open("/user/chae2089/CWpaper/young_kfold_feature6/num_118/mid/report_{}.csv".format(num), 'w') as fp:
            for item in report2_M:
                fp.write("%s\n" % item)
            
        ######### SAVE normal_O    
        result_O=confusion_matrix(test_set_label_O,pred_O)

        filename=("/user/chae2089/CWpaper/young_kfold_feature6/num_118/old/result_%d" %(num))
        np.savetxt(filename, result_O)
        
        report_O = classification_report(test_set_label_O, pred_O)
        report2_O=report_O.split()
        with open("/user/chae2089/CWpaper/young_kfold_feature6/num_118/old/report_{}.csv".format(num), 'w') as fp:
            for item in report2_O:
                fp.write("%s\n" % item)
                 
                
        num = num+1
        